In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import urllib.request
import time
from geopy.geocoders import Nominatim
import geopandas as gpd
import seaborn as sns
import requests
from scipy import stats
from urllib.request import urlopen 
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re

In [8]:
page = requests.get('https://opendata.arcgis.com/datasets/88e8040e02d5493eb163e454140d3a34_0.geojson')
data_crime = pd.read_json(page.content)
data_crime.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'Index_': 8..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'Index_': 8..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'Index_': 8..."
3,FeatureCollection,"{'type': 'Feature', 'properties': {'Index_': 8..."
4,FeatureCollection,"{'type': 'Feature', 'properties': {'Index_': 8..."


In [10]:
data_crime.features['properties']

KeyError: 'properties'

In [ ]:

neighbordhood_list = []
code_postal_list = []
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36 Edg/80.0.100.0'
i=0
for url in data_dict['data_url']:
    url = 'https://www.point2homes.com/'+url
    page = requests.get(url,headers=headers)
    soup = BeautifulSoup(page.content,'html.parser')
    content = soup.find(class_ ='details-charcs')
    if content != None:
        dts = content.find_all('dt')
        dds = content.find_all('dd')
        neig_flag = False;
        code_post_flag = False
        text = ''
        for index in range(len(dts)):
            text = dts[index].get_text().strip()
            if 'Neighborhood' in text:
                neighbordhood_list.append(dds[index].get_text().strip())
                neig_flag = True

            if 'Postal Code' in text:
                code_postal_list.append(dds[index].get_text().strip())
                code_post_flag = True
        if not neig_flag:
            neighbordhood_list.append('--')
        if not code_post_flag:
            code_postal_list.append('--')
    else:
        neighbordhood_list.append('--')
        code_postal_list.append('--')
    time.sleep(1)
    i+=1